## Download and clean up near-real-time Wave Glider data

* read in data
* sort time and remove redundant records
* add lat/lon to ADCP files
* add depth vector to ADCP files
* save cleaned up ADCP files

first cut by Tom, 10/18/2021  
Updated for IOP1, 10/9/2022
Updated version for IOP2, ~4/10/2023

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import cftime
import requests
import cartopy.crs as ccrs                   # import projections
import cartopy
import gsw
import functions  # requires functions.py from this directory

In [2]:
# %matplotlib inline
%matplotlib qt5
plt.rcParams['figure.figsize'] = (7,4)
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 400
plt.close('all')

__figdir__ = '../plots/' 
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
plotfiletype='png'

In [3]:
savefig = True
zoom = True
if zoom:
    xmin, xmax = (-127,-121)
    ymin, ymax = (36.25,38.5)
    levels = np.linspace(14,17,21)-2.5
else:
    xmin, xmax = (-127,-121)
    ymin, ymax = (35, 41)
    levels = np.linspace(13,18,11)

    

Payload 2 Table 1 has met, ctd variables  
Payload 2 Table 2 has RDI variables

In [4]:
# List of WGs
input_list = ['WHOI22','WHOI32','WHOI43','STOKES', 'PLANCK', 'KELVIN', 'CARSON', 'WHOI1102','PASCAL']
url_prefix = 'http://smode.whoi.edu:8080/thredds/fileServer/IOP2_2023/waveglider/'
tab1_postfix = '_21.nc'#PLD2_TAB1
tab2_postfix = '_22.nc' #PLD2_TAB2
position_postfix = '_23.nc' # position??
WG_list = ['WHOI22','WHOI32','WHOI43','STOKES', 'PLANCK', 'KELVIN', 'CARSON', 'WHOI1102','PASCAL']
outpath='../data/raw/WG_NRT_IOP2/'

#http://smode.whoi.edu:8080/thredds/dodsC/IOP2_2023/waveglider/KELVIN_23.nc.html
#http://smode.whoi.edu:8080/thredds/fileServer/IOP2_2023/waveglider/KELVIN_23.nc

In [5]:
# For some reason, reading the files over the internet directly is not working well
# Download instead

n=0
file_list1 = []
file_list2 = []
file_list3 = []
for WG in WG_list:
    input_WG=input_list[n]
    outfile1 = outpath+input_WG+tab1_postfix
    outfile2 = outpath+input_WG+tab2_postfix
    outfile3 = outpath+input_WG+position_postfix
    # Read and save table 1 files
    url1 = url_prefix+input_WG+tab1_postfix
    file_data = requests.get(url1).content
    # create the file in write binary mode, because the data we get from net is in binary
    with open(outfile1, "wb") as file:
        file.write(file_data)
    # Read and save table 2 files
    url2 = url_prefix+input_WG+tab2_postfix
    file_data = requests.get(url2).content
    # create the file in write binary mode, because the data we get from net is in binary
    with open(outfile2, "wb") as file:
        file.write(file_data)
    # Read and save position files
    url3 = url_prefix+input_WG+position_postfix
    file_data = requests.get(url3).content
    # create the file in write binary mode, because the data we get from net is in binary
    with open(outfile3, "wb") as file:
        file.write(file_data)
    n=n+1
    print(url3)
    file_list1.append(outfile1)
    file_list2.append(outfile2)
    file_list3.append(outfile3)


http://smode.whoi.edu:8080/thredds/fileServer/IOP2_2023/waveglider/WHOI22_23.nc
http://smode.whoi.edu:8080/thredds/fileServer/IOP2_2023/waveglider/WHOI32_23.nc
http://smode.whoi.edu:8080/thredds/fileServer/IOP2_2023/waveglider/WHOI43_23.nc
http://smode.whoi.edu:8080/thredds/fileServer/IOP2_2023/waveglider/STOKES_23.nc
http://smode.whoi.edu:8080/thredds/fileServer/IOP2_2023/waveglider/PLANCK_23.nc
http://smode.whoi.edu:8080/thredds/fileServer/IOP2_2023/waveglider/KELVIN_23.nc
http://smode.whoi.edu:8080/thredds/fileServer/IOP2_2023/waveglider/CARSON_23.nc
http://smode.whoi.edu:8080/thredds/fileServer/IOP2_2023/waveglider/WHOI1102_23.nc
http://smode.whoi.edu:8080/thredds/fileServer/IOP2_2023/waveglider/PASCAL_23.nc


In [6]:
file_list2

['../data/raw/WG_NRT_IOP2/WHOI22_22.nc',
 '../data/raw/WG_NRT_IOP2/WHOI32_22.nc',
 '../data/raw/WG_NRT_IOP2/WHOI43_22.nc',
 '../data/raw/WG_NRT_IOP2/STOKES_22.nc',
 '../data/raw/WG_NRT_IOP2/PLANCK_22.nc',
 '../data/raw/WG_NRT_IOP2/KELVIN_22.nc',
 '../data/raw/WG_NRT_IOP2/CARSON_22.nc',
 '../data/raw/WG_NRT_IOP2/WHOI1102_22.nc',
 '../data/raw/WG_NRT_IOP2/PASCAL_22.nc']

In [7]:
def fix_ds_time(ds):
    '''
    Drop nonunique values in realtime data files and sort time.
    
    Input: ds, xarray dataset
    Output: ds, xarray dataset
    '''
    t, ind = np.unique(ds.time, return_index=True)
    ds2 = ds.isel(time=ind,drop=True)
    return ds2

In [8]:
# Read in files (Payload 2 Tables 1 and 2; position) from all WG
n=0
for WG in WG_list:
    input_WG=input_list[n]
    file1 = file_list1[n]
    file2 = file_list2[n]
    file3 = file_list3[n]
    varstr = 'met_'+WG
    ds_met_temp=xr.open_dataset(file1,decode_times=True)
    locals()[varstr]=fix_ds_time(ds_met_temp) #Drop nonunique values and sort time
    varstr = 'adcp_'+WG
    !ncrename -v z,z_matrix $file2 #renaming variable z to prevent dimension/variable name conflict in xarray, requires nco in linux
    ds_adcp_temp=xr.open_dataset(file2,decode_times=True)
    locals()[varstr]=fix_ds_time(ds_adcp_temp) #Drop nonunique values and sort time
    varstr = 'wave_'+WG
    ds_pos_temp=xr.open_dataset(file3,decode_times=True)
    locals()[varstr]=fix_ds_time(ds_pos_temp) #Drop nonunique values and sort time
    n=n+1
    print(file1)

ncrename: ERROR Required variable 'z' is not present in input file. HINT: If presence is intended to be optional, then prefix old variable name with the period character '.', i.e., 'ncrename -v .z,z_matrix'. With this syntax ncrename would succeed even when no such variable is in the file.
../data/raw/WG_NRT_IOP2/WHOI22_21.nc
ncrename: ERROR Required variable 'z' is not present in input file. HINT: If presence is intended to be optional, then prefix old variable name with the period character '.', i.e., 'ncrename -v .z,z_matrix'. With this syntax ncrename would succeed even when no such variable is in the file.
../data/raw/WG_NRT_IOP2/WHOI32_21.nc
ncrename: ERROR Required variable 'z' is not present in input file. HINT: If presence is intended to be optional, then prefix old variable name with the period character '.', i.e., 'ncrename -v .z,z_matrix'. With this syntax ncrename would succeed even when no such variable is in the file.
../data/raw/WG_NRT_IOP2/WHOI43_21.nc
ncrename: ERROR 

In [9]:
eval('met_'+WG)

<xarray.Dataset>
Dimensions:                      (time: 2032)
Coordinates:
  * time                         (time) datetime64[ns] 2023-04-17T00:01:00 .....
Data variables: (12/68)
    cr6_card_status_flag         (time) int32 ...
    batt_volt_Min                (time) float64 ...
    panel_temp_Avg               (time) float64 ...
    proc_time_Max                (time) float64 ...
    proc_time_Avg                (time) float64 ...
    slow_proc_time_1_Max         (time) float64 ...
    ...                           ...
    sig_pressure_Avg             (time) float64 ...
    sig_temperature_Avg          (time) float64 ...
    index_1hz_file               (time) int32 ...
    index_sbg_file               (time) int32 ...
    index_adcp_file              (time) int32 ...
    index_sig_file               (time) int32 ...
Attributes:
    title:                S-MODE IOP2 2023 wave glider
    summary:              real-time wave glider data recorded by SV3 during S...
    time_coverage_start:  2023-04-17T00:01:00Z
    time_coverage_end:    2023-05-08T21:46:00Z

In [10]:
eval('adcp_'+WG)

<xarray.Dataset>
Dimensions:                (time: 2049, depth: 50)
Coordinates:
  * time                   (time) datetime64[ns] 2023-04-17T00:02:00 ... 2023...
  * depth                  (depth) float64 -3.93 -5.93 -7.93 ... -99.93 -101.9
Data variables:
    adcp_roll_inst_Avg     (time) float64 ...
    adcp_pitch_inst_Avg    (time) float64 ...
    adcp_heading_inst_Avg  (time) float64 ...
    adcp_temperature_Avg   (time) float64 ...
    true_current_east      (time, depth) float64 ...
    true_current_north     (time, depth) float64 ...
Attributes:
    title:                S-MODE IOP2 2023 wave glider
    summary:              real-time wave glider data recorded by SV3 during S...
    time_coverage_start:  2023-04-17T00:02:00Z
    time_coverage_end:    2023-05-08T21:47:00Z

In [11]:
eval('wave_'+WG)

<xarray.Dataset>
Dimensions:  (time: 2054, frequency: 50, freq: 50)
Coordinates:
  * time     (time) datetime64[ns] 2023-04-17T00:03:00 ... 2023-05-08T21:48:00
  * freq     (freq) float64 0.03906 0.04883 0.05859 ... 0.7324 0.7617 0.8008
Dimensions without coordinates: frequency
Data variables:
    f2xSuu   (time, frequency) float64 ...
    theta    (time, frequency) float64 ...
Attributes:
    title:                S-MODE IOP2 2023 wave glider
    summary:              real-time wave glider data recorded by SV3 during S...
    time_coverage_start:  2023-04-17T00:03:00Z
    time_coverage_end:    2023-05-08T21:48:00Z

In [12]:
# Write WHOI22 met record to file
# met_WHOI22.to_netcdf('../data/raw/WG_NRT/WHOI22_met.nc')


In [13]:
# Now we can access these in a loop using syntax like:
# eval('adcp_'+WG_list[7])

In [14]:
eval('met_'+WG_list[0])

<xarray.Dataset>
Dimensions:                      (time: 4050)
Coordinates:
  * time                         (time) datetime64[ns] 2023-03-23T20:31:00 .....
Data variables: (12/56)
    cr6_card_status_flag         (time) int32 ...
    batt_volt_Min                (time) float64 ...
    panel_temp_Avg               (time) float64 ...
    proc_time_Max                (time) float64 ...
    proc_time_Avg                (time) float64 ...
    slow_proc_time_1_Max         (time) float64 ...
    ...                           ...
    lctd_press_Avg               (time) float64 ...
    lctd_temp_Avg                (time) float64 ...
    lctd_cond_Avg                (time) float64 ...
    index_1hz_file               (time) int32 ...
    index_sbg_file               (time) int32 ...
    index_adcp_file              (time) int32 ...
Attributes:
    title:                S-MODE IOP2 2023 wave glider
    summary:              real-time wave glider data recorded by SV3 during S...
    time_coverage_start:  2023-03-23T20:31:00Z
    time_coverage_end:    2023-05-08T22:01:00Z

In [15]:
#Compute density from T and cond
p = 1
for WG in WG_list:
    ds = eval('met_'+WG)
    ds['uctd_psu_Avg']=gsw.conversions.SP_from_C(10*ds.uctd_cond_Avg, ds.uctd_temp_Avg, p)
    SA = gsw.conversions.SA_from_SP(ds.uctd_psu_Avg, 1,ds.longitude_1hz_Avg, ds.latitude_1hz_Avg)
    CT = gsw.conversions.CT_from_t(SA, ds.uctd_temp_Avg, p)
    ds['uctd_sigma0_Avg'] = gsw.density.sigma0(SA, CT)
    varstr = 'met_'+WG
    locals()[varstr]= ds

OK, now let's look at RDI files (Table 2)

OK, we have 15 minute files from the ADCP and 5 minute from the position files.  Interpolate the position files to the ADCP times.  That should be easy using xarray interp package, following:  
https://docs.xarray.dev/en/stable/user-guide/interpolation.htmlhttps://docs.xarray.dev/en/stable/user-guide/interpolation.html  

```
new_lon = -126.1
new_lat = 37.1
new_time = ds.time[-3]
dsi = ds.interp(time=new_time,latitude=new_lat, longitude=new_lon)
```

```
new_time = ds_adcp.time
ds_pos_i = ds_pos.interp(time=new_time)
```

In [16]:
# Interpolate each WG's position to ADCP time and add to ADCP file
for WG in WG_list:
    ds_adcp = eval('adcp_'+WG)
    ds_pos = eval('met_'+WG)
    ds_pos_i = ds_pos.interp(time=ds_adcp.time)
    ds_adcp['Longitude']=ds_pos_i.longitude_1hz_Avg
    ds_adcp['Latitude']=ds_pos_i.latitude_1hz_Avg
    varstr = 'adcp_'+WG
    locals()[varstr]= ds_adcp
    del ds_adcp

OK, that's very cool!  I have all the files cleaned up and have added the lat/lon.  Let's save the cleaned up files for met and adcp.  First, add z for adcp files.

In [17]:
for WG in WG_list:
    fout_adcp = outpath + 'adcp_'+WG + '.nc'
    ds_adcp = eval('adcp_'+WG)
    ds_adcp = ds_adcp.rename(name_dict={'true_current_north':'current_north', 'true_current_east':'current_east'})
    ds_adcp.to_netcdf(fout_adcp)

    fout_met = outpath + 'met_'+WG + '.nc'
    ds_met = eval('met_'+WG)
    ds_met.to_netcdf(fout_met)



In [18]:
%whos

Variable           Type              Data/Info
----------------------------------------------
CT                 DataArray         <xarray.DataArray 'uctd_p<...>0 ... 2023-05-08T21:46:00
SA                 DataArray         <xarray.DataArray 'uctd_p<...>0 ... 2023-05-08T21:46:00
WG                 str               PASCAL
WG_list            list              n=9
adcp_CARSON        Dataset           <xarray.Dataset>\nDimensi<...>:    2023-04-28T20:32:00Z
adcp_KELVIN        Dataset           <xarray.Dataset>\nDimensi<...>:    2023-05-08T21:47:00Z
adcp_PASCAL        Dataset           <xarray.Dataset>\nDimensi<...>:    2023-05-08T21:47:00Z
adcp_PLANCK        Dataset           <xarray.Dataset>\nDimensi<...>:    2023-05-08T22:02:00Z
adcp_STOKES        Dataset           <xarray.Dataset>\nDimensi<...>:    2023-05-05T19:02:00Z
adcp_WHOI1102      Dataset           <xarray.Dataset>\nDimensi<...>:    2023-05-08T22:02:00Z
adcp_WHOI22        Dataset           <xarray.Dataset>\nDimensi<...>:    2023-

In [19]:
ds_adcp

<xarray.Dataset>
Dimensions:                (time: 2049, depth: 50)
Coordinates:
  * time                   (time) datetime64[ns] 2023-04-17T00:02:00 ... 2023...
  * depth                  (depth) float64 -3.93 -5.93 -7.93 ... -99.93 -101.9
Data variables:
    adcp_roll_inst_Avg     (time) float64 ...
    adcp_pitch_inst_Avg    (time) float64 ...
    adcp_heading_inst_Avg  (time) float64 ...
    adcp_temperature_Avg   (time) float64 ...
    current_east           (time, depth) float64 ...
    current_north          (time, depth) float64 ...
    Longitude              (time) float64 -124.0 -124.0 -124.0 ... -123.9 nan
    Latitude               (time) float64 36.79 36.76 36.74 ... 36.88 36.88 nan
Attributes:
    title:                S-MODE IOP2 2023 wave glider
    summary:              real-time wave glider data recorded by SV3 during S...
    time_coverage_start:  2023-04-17T00:02:00Z
    time_coverage_end:    2023-05-08T21:47:00Z

In [20]:
vmin = -0.5
vmax = 0.5
fig = plt.figure()
plt.set_cmap(cmap=plt.get_cmap('turbo'))
# ax1 = plt.subplot(len(WG_list),1,len(WG_list))
# ax1.set_xlim(tmin,tmax)
ds = adcp_CARSON
im = plt.pcolor(ds.time.values,ds.depth,ds.true_current_north.transpose(),vmin=vmin,vmax=vmax)
# plt.contourf(ds.time.values,ds.z_matrix[:,1],ds.current_east,levels)
plt.ylim(-60, 0)
plt.title(' Carson North vel')
fig=plt.gcf()
fig.autofmt_xdate()


In [21]:
pwd

'/home/jtomf/Python/S-MODE_analysis/code_IOP2'

In [22]:
z0 = -4
zind = np.flatnonzero(np.abs(ds.depth-z0)<1)
ds = adcp_WHOI1102
tmin = np.datetime64('2023-04-17T10:00:00')#'2023-04-21T10:00:00')
tmax = np.datetime64('now')#np.datetime64('2023-04-22T09:00:00')
vmin = -0.5
vmax = 0.5
fig, ax = plt.subplots(3,1,figsize=(8, 6),sharex=True)
ax[0].plot(ds.time,ds.true_current_east[:,zind])
ax[0].set_ylim(-.5, -0)
ax[0].set(ylabel='4-m vel [m/s]')
ax[2].plot(ds.time,ds.Longitude)
ax[2].set_ylim(-124.15, -123.9)
ax[2].set(ylabel='longitude')

plt.set_cmap(cmap=plt.get_cmap('turbo'))
# ax1 = plt.subplot(len(WG_list),1,len(WG_list))
# ax1.set_xlim(tmin,tmax)
im = ax[1].pcolor(ds.time.values,ds.depth,ds.true_current_east.transpose(),vmin=vmin,vmax=vmax)
# plt.contourf(ds.time.values,ds.z_matrix[:,1],ds.current_east,levels)
ax[1].set_ylim(-60, 0)
ax[1].set_xlim(left=tmin, right=tmax)

ax[0].set_title('WHOI1102 East vel')
fig=plt.gcf()
fig.autofmt_xdate()
